
# [FuzzyWuzzyProto] Jupyter notebook

# The problem :

A list of names has been generated from our database, where although the IDs match for each row, the names do not match.
This is due to a variety of factors, such as illegal characters or the names being in reverse order, or in some cases an email address appearing in the name field.
The purpose of this notebook is to try and improve the similarity of the match using both data cleaning steps in python and fuzzy matching.

Note: This notebook has been updated to use the Room Type Data from Kaggle, as the original matching exercise contained personally identifiable information. Thanks to the team who assembled and provided the data for use.
https://www.kaggle.com/leandrodoze/room-type 


# The packages :

First things first, let's import `Pandas` :

In [1]:
import pandas as pd
import numpy as np

If you don't have `FuzzyWuzzy` installed, you can install it by running `pip install fuzzywuzzy[speedup]` inside your Jupyter notebook or from the PIP or Anaconda prompt. If this doesn't work, you should install `FuzzyWuzzy` like this: `pip install fuzzywuzzy` and then install `python-levenshtein` like this: `pip install python-Levenshtein` (in order to speed the matching process).

From `FuzzyWuzzy`, we'll mainly need two modules: `process` and `fuzz` :

In [2]:
from fuzzywuzzy import process, fuzz

Let's also import `matplotlib` for data visualization and also some modules to improve readability of the notebook:

In [3]:
import matplotlib.pyplot as plt
from IPython.display import HTML, display, Math, Latex, Image
HTML('''<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-MML-AM_CHTML' async></script>''')

 And for good measure, let's add some CSS to style our dataframes for added contrast between the axes and the data itself (If you don't like this style, just delete the cell, restart the kernel and re-run all cells). <br>
<sub id='codex'>[code explanation 1](#codex1)</sub>
<a id='code1'></a>

In [4]:
%%HTML
<style>.dataframe th, td:first-child{background:#3f577c;font-family:monospace;color:white;border:3px solid white;
text-align:left !important;}#codex{float:right;}</style>

This next bit just insures the custom CSS style we just applied stays on when we use `style.background_gradient` later on:

In [5]:
th_props = [('background','#3f577c'), ('font-family','monospace'), ('color','white'),('border','3px solid white'),
            ('text-align','left !important')]
styles = [dict(selector="th", props=th_props)]

# The data :

We have two dataframes or tables we'll need to match here :
* `example_name_data_a` that contains first name and last name data for one group of students;
* `example_name_data_b` that contains first name and last name data for another group of students;


In [6]:
a_df = pd.read_csv('data/example_name_data_a.csv', encoding='unicode_escape')
b_df = pd.read_csv('data/example_name_data_b.csv', encoding='unicode_escape')
display( a_df.head(10) )
display( b_df.head(10) )

,SFDC_ACCOUNT_ID,name_a
0,0012400000qIKr5AAG,"Deluxe Room, 1 King Bed"
1,0012400000qJfleAAC,"Standard Room, 1 King Bed, Accessible"
2,0012400000qJfp2AAC,"Grand Corner King Room, 1 King Bed"
3,0012400000vZcUxAAK,"Suite, 1 King Bed (Parlor)"
4,0011p00001YEVvfAAH,"High-Floor Premium Room, 1 King Bed"
5,0012400001Ko0VnAAJ,"Traditional Double Room, 2 Double Beds"
6,0012400000J4kLeAAJ,"Room, 1 King Bed, Accessible"
7,0011p00001anUF3AAM,"Deluxe Room, 1 King Bed"
8,0011p00001Ti4LkAAJ,Deluxe Room
9,0014H000021UEyqQAG,"Room, 2 Double Beds (19th to 25th Floors)"


,SFDC_ACCOUNT_ID,name_b
0,0012400000qIKr5AAG,Deluxe King Room
1,0012400000qJfleAAC,Standard King Roll-in Shower Accessible
2,0012400000qJfp2AAC,Grand Corner King Room
3,0012400000vZcUxAAK,King Parlor Suite
4,0011p00001YEVvfAAH,High-Floor Premium King Room
5,0012400001Ko0VnAAJ,Double Room with Two Double Beds
6,0012400000J4kLeAAJ,King Room - Disability Access
7,0011p00001anUF3AAM,Deluxe King Room
8,0011p00001Ti4LkAAJ,Deluxe Room (Non Refundable)
9,0014H000021UEyqQAG,Two Double Beds - Location Room (19th to 25th ...


---

# Data Cleaning :

Before using `FuzzyWuzzy`, we need to clean our data frames. 
The idea is to :

1. Remove illegal characters or incorrect values



In [7]:
REQUIRED_FIELDS_a = ['name_a']
REQUIRED_FIELDS_b = ['name_b']

In [8]:
a_test_df = a_df
a_test_df.fillna('', inplace=True)
b_test_df = b_df
b_test_df.fillna('', inplace=True)
display( a_test_df.head(10) )
display( b_test_df.head(10) )

,SFDC_ACCOUNT_ID,name_a
0,0012400000qIKr5AAG,"Deluxe Room, 1 King Bed"
1,0012400000qJfleAAC,"Standard Room, 1 King Bed, Accessible"
2,0012400000qJfp2AAC,"Grand Corner King Room, 1 King Bed"
3,0012400000vZcUxAAK,"Suite, 1 King Bed (Parlor)"
4,0011p00001YEVvfAAH,"High-Floor Premium Room, 1 King Bed"
5,0012400001Ko0VnAAJ,"Traditional Double Room, 2 Double Beds"
6,0012400000J4kLeAAJ,"Room, 1 King Bed, Accessible"
7,0011p00001anUF3AAM,"Deluxe Room, 1 King Bed"
8,0011p00001Ti4LkAAJ,Deluxe Room
9,0014H000021UEyqQAG,"Room, 2 Double Beds (19th to 25th Floors)"


,SFDC_ACCOUNT_ID,name_b
0,0012400000qIKr5AAG,Deluxe King Room
1,0012400000qJfleAAC,Standard King Roll-in Shower Accessible
2,0012400000qJfp2AAC,Grand Corner King Room
3,0012400000vZcUxAAK,King Parlor Suite
4,0011p00001YEVvfAAH,High-Floor Premium King Room
5,0012400001Ko0VnAAJ,Double Room with Two Double Beds
6,0012400000J4kLeAAJ,King Room - Disability Access
7,0011p00001anUF3AAM,Deluxe King Room
8,0011p00001Ti4LkAAJ,Deluxe Room (Non Refundable)
9,0014H000021UEyqQAG,Two Double Beds - Location Room (19th to 25th ...


In [9]:
a_test_df = a_test_df.replace(to_replace=r'(\#)|(\?+)|(UNKNOWN)|(unknown)|(student)|(STUDENT)', value='', regex=True)
a_test_df = a_test_df.replace(to_replace=r'(\s\-)|(\s\.)|(\.\s)|(\,\s)|(\s\.\s)|(\-\s)|(\s\-)|(\s\:)|(\s\_)|(\:)|(\.)|(\_)|(\/)|(\-)|(\")|(\,)|(\s{2,})', value=' ', regex=True)
#print(clean_field)
#print(pd.Series(test_df[clean_field]))
display( a_test_df.head(10) )

,SFDC_ACCOUNT_ID,name_a
0,0012400000qIKr5AAG,Deluxe Room 1 King Bed
1,0012400000qJfleAAC,Standard Room 1 King Bed Accessible
2,0012400000qJfp2AAC,Grand Corner King Room 1 King Bed
3,0012400000vZcUxAAK,Suite 1 King Bed (Parlor)
4,0011p00001YEVvfAAH,High Floor Premium Room 1 King Bed
5,0012400001Ko0VnAAJ,Traditional Double Room 2 Double Beds
6,0012400000J4kLeAAJ,Room 1 King Bed Accessible
7,0011p00001anUF3AAM,Deluxe Room 1 King Bed
8,0011p00001Ti4LkAAJ,Deluxe Room
9,0014H000021UEyqQAG,Room 2 Double Beds (19th to 25th Floors)


In [10]:
b_test_df = b_test_df.replace(to_replace=r'(\#)|(\?+)|(UNKNOWN)|(unknown)|(student)|(STUDENT)', value='', regex=True)
b_test_df = b_test_df.replace(to_replace=r'(\s\-)|(\s\.)|(\.\s)|(\,\s)|(\s\.\s)|(\-\s)|(\s\-)|(\s\:)|(\s\_)|(\:)|(\.)|(\_)|(\/)|(\-)|(\")|(\,)|(\s{2,})', value=' ', regex=True)
#print(clean_field)
#print(pd.Series(test_df[clean_field]))
display( b_test_df.head(10) )

,SFDC_ACCOUNT_ID,name_b
0,0012400000qIKr5AAG,Deluxe King Room
1,0012400000qJfleAAC,Standard King Roll in Shower Accessible
2,0012400000qJfp2AAC,Grand Corner King Room
3,0012400000vZcUxAAK,King Parlor Suite
4,0011p00001YEVvfAAH,High Floor Premium King Room
5,0012400001Ko0VnAAJ,Double Room with Two Double Beds
6,0012400000J4kLeAAJ,King Room Disability Access
7,0011p00001anUF3AAM,Deluxe King Room
8,0011p00001Ti4LkAAJ,Deluxe Room (Non Refundable)
9,0014H000021UEyqQAG,Two Double Beds Location Room (19th to 25th F...


In [11]:
for clean_field in REQUIRED_FIELDS_a:
    #clean_field = clean_field.strip().replace(' ', '').lower()
    a_test_df[clean_field] = a_test_df[clean_field].str.strip().str.lower()


In [12]:
for clean_field in REQUIRED_FIELDS_b:
    #clean_field = clean_field.strip().replace(' ', '').lower()
    b_test_df[clean_field] = b_test_df[clean_field].str.strip().str.lower()

In [13]:
#drop rows where the matching field is empty after cleaning
#a_test_df.drop(a_test_df[a_test_df['name_a'] == ''].index, inplace=True)
#b_test_df.drop(b_test_df[b_test_df['name_b'] == ''].index, inplace=True)


In [14]:
#a_test_df = a_test_df.sort_values(by=['name_a']).reset_index(drop=True)
#b_test_df = b_test_df.sort_values(by=['name_b']).reset_index(drop=True)
display( a_test_df.head(3) )
display( b_test_df.head(3) )

,SFDC_ACCOUNT_ID,name_a
0,0012400000qIKr5AAG,deluxe room 1 king bed
1,0012400000qJfleAAC,standard room 1 king bed accessible
2,0012400000qJfp2AAC,grand corner king room 1 king bed


,SFDC_ACCOUNT_ID,name_b
0,0012400000qIKr5AAG,deluxe king room
1,0012400000qJfleAAC,standard king roll in shower accessible
2,0012400000qJfp2AAC,grand corner king room


# What is the FuzzyWuzzy Package? :

The `FuzzyWuzzy` package compares two strings A and B and outputs a ratio that estimates the Levenshtein distance between them. The Levenshtein distance, which is the distance between A and B in terms of how many changes we have to make to the string A in order to transform it into string B. The changes include removing, adding or substituting characters. The fewer the changes we have to make, the more similar A and B are, which results in a higher ratio.

As part of this prototype, I have been comparing multiple methods taken from articles to investigate different methods which could be productionised.

# FuzzyWuzzy application :

### Method 1:
Pairs of names given a specific column in each file

#### Problem: not productionised as a function!

#### Example: </br>
* Row 30 from both files:
* abdallah allidawi
* abdullah alliddawi
* ratio = 91

In [15]:
#Row 30 from both files - simple 1 letter difference
print(a_test_df['name_a'][30])
print(b_test_df['name_b'][30])
print ('ratio =' , fuzz.ratio(a_test_df['name_a'][30],b_test_df['name_b'][30]))

executive room 1 king bed non smoking
executive king room
ratio = 61


#### Method 2: 
* Create tuples as a series, and then process all match possibilities from both files
* https://www.semicolonworld.com/question/58508/apply-fuzzy-matching-across-a-dataframe-column-and-save-results-in-a-new-column

#### Problem 1: Really low performance
#### Problem 2: Duplicates create a problem
* 'ValueError: Index contains duplicate entries, cannot reshape'

In [16]:
compare = pd.MultiIndex.from_product([a_test_df['name_a'],
                                      b_test_df['name_b']]).to_series()

In [17]:
def metrics(tup):
    return pd.Series([fuzz.ratio(*tup),
                      fuzz.token_sort_ratio(*tup)],
                     ['ratio', 'token'])

In [ ]:
compare.apply(metrics)

In [ ]:
compare.apply(metrics).unstack().idxmax().unstack(0)

In [ ]:
compare.apply(metrics).unstack(0).idxmax().unstack(0)

### Method 3: 
* Create a function using process.extract
* Based on: https://medium.com/analytics-vidhya/matching-messy-pandas-columns-with-fuzzywuzzy-4adda6c7994f
* and https://github.com/kelmouloudi/MediumArticle-FuzzyWuzzy

#### Problem: AGAIN low performance
* Work fine for two files with 100 rows
* Works slowly for two files with 5k rows!

In [20]:
a_test_df = a_test_df.sort_values(by=['name_a']).reset_index(drop=True)
b_test_df = b_test_df.sort_values(by=['name_b']).reset_index(drop=True)
# ideal to sort first for this function, performs better
# index must be correct for both to work properly

match_b = []
similarity = []

for i in a_test_df.name_a:
        ratio = process.extract( i, b_test_df.name_b, limit=1)
        match_b.append(ratio[0][0])
        similarity.append(ratio[0][1])

a_test_df['match_b'] = pd.Series(match_b)
a_test_df['similarity'] = pd.Series(similarity)

a_test_df.head(10)

,SFDC_ACCOUNT_ID,name_a,match_b,similarity
0,0012400000l6u0AAAQ,business double room 2 double beds,business double room with two double beds,91
1,0012400000jqfYCAAY,business plan 1 king bed,accessible club ocean view suite with one king...,86
2,00124000019HHXlAAO,city room city view,queen room with city view,95
3,0014H00002GHEvcQAH,classic room 1 king bed,classic king room,95
4,00124000008FHqrAAG,club room 1 king bed,accessible club ocean view suite with one king...,86
5,0011p00002JYCDNAA5,club room 1 king bed,accessible club ocean view suite with one king...,86
6,0011p0000265zAlAAI,club room 1 king bed oceanfront,accessible club ocean view suite with one king...,86
7,0012400001HOFKEAA5,club room 2 queen beds,accessible club ocean view suite with one king...,86
8,0011p00001dvdacAAA,club room 2 queen beds,accessible club ocean view suite with one king...,86
9,0012400000bjZrqAAE,club room city view (club lounge access for 2 ...,business king room,86


Now simplify this dataframe by only keeping the name and and `similarity` columns. Let's call it `final_result` : 

In [21]:
# Results Table: 100% similarity
final_result_100 = a_test_df[a_test_df.similarity==100]
final_result_100 = final_result_100.sort_values(by=['SFDC_ACCOUNT_ID']).reset_index(drop=True)
final_result_100

## TODO ##
# Add a 'Match_Type' column which indicates these are "definite" matches

,SFDC_ACCOUNT_ID,name_a,match_b,similarity
0,0012400001Ime60AAB,regency club mountain view,regency club mountain view,100
1,0012400001Ime94AAB,regency club ocean view,regency club ocean view,100
2,0014H000021VWCKQA4,deluxe suite,deluxe suite,100


In [22]:
# Results Table: 90-99% similarity

final_result_90 = a_test_df[(a_test_df.similarity < 100) & (a_test_df.similarity > 89)]
final_result_90 = final_result_90.sort_values(by=['similarity','SFDC_ACCOUNT_ID']).reset_index(drop=True)
final_result_90

## TODO ##
# Add a 'Match_Type' column which indicates these are "Highly Likely Potential" matches
# change to descending order by similarity, ascending by account ID

,SFDC_ACCOUNT_ID,name_a,match_b,similarity
0,0011p00001ft73NAAQ,junior suite 1 king bed with sofa bed,junior suite,90
1,0011p00002OYDpYAAX,double room,business double room with two double beds,90
2,0012400000aAGLxAAO,deluxe suite 1 bedroom,deluxe one bedroom suite,90
3,0012400000qJfp2AAC,grand corner king room 1 king bed,grand corner king room,90
4,0012400001Ow1FQAAZ,king room suite 1 king bed with sofa bed,king room,90
5,0014H00001wABGcQAO,deluxe suite 1 king bed non smoking kitchen,deluxe suite,90
6,0011p00001ZrpzsAAB,room 1 king bed ocean view,ocean view room with king bed,91
7,0012400000GGZBZAA5,deluxe room 2 queen beds,deluxe room two queen beds,91
8,0012400000l6u0AAAQ,business double room 2 double beds,business double room with two double beds,91
9,0014H000021UEyqQAG,room 2 double beds (19th to 25th floors),two double beds location room (19th to 25th f...,92


In [23]:
# Results Table: 80-90% similarity
final_result_80 = a_test_df[(a_test_df.similarity < 90) & (a_test_df.similarity > 79)]
final_result_80 = final_result_80.sort_values(by=['similarity']).reset_index(drop=True)
final_result_80

## TODO ##
# Add a 'Match_Type' column which indicates these are "Likely Potential" matches
# change to descending order by similarity, ascending by account ID

,SFDC_ACCOUNT_ID,name_a,match_b,similarity
0,0012400000jqfYCAAY,business plan 1 king bed,accessible club ocean view suite with one king...,86
1,0014H00002WncdmQAB,room 1 king bed pool view,accessible club ocean view suite with one king...,86
2,0011p00001ZrpzsAAB,room 1 king bed resort view (alii),accessible club ocean view suite with one king...,86
3,0011p00002OYDJAAA5,room 1 queen bed,accessible club ocean view suite with one king...,86
4,0014H00002O27MAQAZ,room 1 queen bed accessible bathtub,alii tower resort view with king bed mobility...,86
5,00124000015GfohAAC,room 1 queen bed city view,accessible club ocean view suite with one king...,86
6,0012400000BFtEaAAL,room 1 queen bed non smoking (fairmont room),business king room,86
7,0012400001Ow1FQAAZ,room 2 double beds,accessible partial ocean view with two double ...,86
8,0012400000YdK0fAAF,room 2 double beds city view,accessible club ocean view suite with one king...,86
9,0014H00002O21tyQAB,room 2 double beds non smoking,accessible partial ocean view with two double ...,86


In [24]:
# Results Tables - low or no similarity
final_result_low_similarity = a_test_df[(a_test_df.similarity < 80) ]
final_result_low_similarity = final_result_low_similarity.sort_values(by=['similarity']).reset_index(drop=True)
final_result_low_similarity

## TODO ##
# Add a 'Match_Type' column which indicates these are "Not Likely Potential" matches

,SFDC_ACCOUNT_ID,name_a,match_b,similarity


In [25]:
# Expected Output columns same as settlement file
# Same Mandatory files
# And same output structure

### Method 4: [DO NOT USE!]
* Use a find match function and process.extractOne
* Like this methods THE LEAST
* https://stackoverflow.com/questions/53103848/fuzzy-matching-two-columns-in-the-same-dataframe-using-python


#### Problem: AGAIN low performance
Work fine for two files with 100 rows
Works horribly for two files with 5k rows!
Do not like the output - it's not intuitive

In [ ]:
#import pandas as pd
#from fuzzywuzzy import process, fuzz

name_list = b_test_df['name_b']

def find_match(x):

    match = process.extractOne(x['name_a'], name_list, scorer=fuzz.ratio)
    return pd.Series([match[0], match[1]])

a_test_df[['match','match_rating']] = a_test_df.apply(find_match, axis=1, result_type='expand')
a_test_df[['match','match_rating']]

### METHOD 5
* Fuzzy Merge Function to produce desired output
* works pretty well and produces a good output
* not tested on larger data sets yet
https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas


In [26]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [27]:
fuzzy_merge(a_test_df, b_test_df, 'name_a', 'name_b', threshold=80)

,SFDC_ACCOUNT_ID,name_a,match_b,similarity,matches
0,0012400000l6u0AAAQ,business double room 2 double beds,business double room with two double beds,91,"business double room with two double beds, bus..."
1,0012400000jqfYCAAY,business plan 1 king bed,accessible club ocean view suite with one king...,86,accessible club ocean view suite with one king...
2,00124000019HHXlAAO,city room city view,queen room with city view,95,"queen room with city view, room with city view"
3,0014H00002GHEvcQAH,classic room 1 king bed,classic king room,95,"classic king room, accessible club ocean view ..."
4,00124000008FHqrAAG,club room 1 king bed,accessible club ocean view suite with one king...,86,accessible club ocean view suite with one king...
5,0011p00002JYCDNAA5,club room 1 king bed,accessible club ocean view suite with one king...,86,accessible club ocean view suite with one king...
6,0011p0000265zAlAAI,club room 1 king bed oceanfront,accessible club ocean view suite with one king...,86,accessible club ocean view suite with one king...
7,0012400001HOFKEAA5,club room 2 queen beds,accessible club ocean view suite with one king...,86,accessible club ocean view suite with one king...
8,0011p00001dvdacAAA,club room 2 queen beds,accessible club ocean view suite with one king...,86,accessible club ocean view suite with one king...
9,0012400000bjZrqAAE,club room city view (club lounge access for 2 ...,business king room,86,"business king room, city view with one king bed"


# Conclusions
Of the 5 methods tested, some were a better fir for our needs, and some were either a very poor fit or took a very long time to run.

Method 2 and Method 4 had quite low performance on the original data set, and 4 produces a large output that isn't easy to deal with.
Method 1 is a bit too simplistic.

The recommended methods would be 3 (although not on a large data set) or 5 if you know the threshold for matching that you wish to use.